## Experimental results : denoising a known covariance matrix

We generate a vector of means and a covariance matrix out of 10 blocks of size 50
each, where off-diagonal elements within each block have a correlation of 0.5.

- This covariance matrix is a stylized representation of a “true” (non-empirical) de-toned correlation
matrix of the S&P 500, where each block is associated with an economic sector.

- Without loss of generality, the variances are drawn from a uniform distribution bounded between
5% and 20%, and the vector of means is drawn from a Normal distribution with mean and standard
deviation equal to the standard deviation from the covariance matrix.

In [2]:
# Import library
import numpy as np
from scipy.linalg import block_diag

#### The real covariance matrix

In [3]:
# Generate our covariance matrix
Nb_blocks = 10
block_size = 50
block_list = []
for i in range(Nb_blocks):
    sd = np.sqrt(np.random.uniform(0.05,0.2,50))
    sd = sd[:,np.newaxis]
    C = sd @ np.transpose(sd) 
    diag = np.diag(C)
    C = C * 0.5
    np.fill_diagonal(C, diag)
    block_list.append(C)

Cov = block_diag(*block_list)


In [4]:
# Generate mean
sd = np.sqrt(np.diag(Cov))
mu = np.random.multivariate_normal(sd, Cov)

In [17]:
# The minimum-variance weight 
ones = np.ones((500,1))
w = np.linalg.inv(Cov) @ ones / (np.transpose(ones) @ np.linalg.inv(Cov) @ ones)

#### The estimated covariance matrix with noise

In [5]:
# Generate observation matrix X
T = 1000
X = np.random.multivariate_normal(mu, Cov, T)

In [11]:
# Estimation without denoising
mu_hat = np.mean(X, axis = 0)
mu_hat = mu_hat[:,np.newaxis]
Cov_hat = 1 / T * np.transpose(X) @ X - mu_hat @ np.transpose(mu_hat)

In [19]:
w_hat = np.linalg.inv(Cov_hat) @ ones / (np.transpose(ones) @ np.linalg.inv(Cov_hat) @ ones)

In [22]:
np.sqrt(np.mean((w_hat-w)**2))

0.008789357085477147

#### The denoised covariance matrix